### Dynamic breakpoints

Objetivos
Os pontos de interrupção são definidos pelo desenvolvedor em um nó específico durante a compilação do grafo.

Mas, às vezes, é útil permitir que o grafo se interrompa dinamicamente!

Este é um ponto de interrupção interno e pode ser implementado usando NodeInterrupt.

Isso oferece alguns benefícios específicos:

(1) Você pode fazer isso condicionalmente (de dentro de um nó, com base na lógica definida pelo desenvolvedor).

(2) Você pode comunicar ao usuário o motivo da interrupção (passando o que quiser para o NodeInterrupt).

Vamos criar um grafo onde um NodeInterrupt é lançado com base no comprimento da entrada.

In [ ]:
from IPython.display import Image, display

from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import NodeInterrupt
from langgraph.graph import START, END, StateGraph


class State(typedDict):
    input: str

def step1(state: State) -> State:
    print("----Step 1----")
    return state

def step2(state: State)-> State:
    # Opcionalmente, vamos gerar uma NodeInterrupt se o comprimento da entrada for maior que 5 caracteres.
    if len(state['input']) > 5:
        raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")
    
    print("---Step 2---")
    return state

def step3(state: State)-> State:
    print("---Step 3---")
    return state


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Compile the graph with memory
graph = builder.compile(checkpointer=memory)

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
initial_input = {"input": "hello world"}
thread_config = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread_config, stream_mode="values"):
    print(event)

Atualizando o state

In [ ]:
graph.update_state(
    thread_config,
    {"input": "hi"},
)

In [ ]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)